In [1]:
import torch
from transformers.models.bert import BertModel, BertTokenizer

In [2]:
model_name = 'bert-base-uncased'

In [3]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# "intermediate_size": 3072,
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [6]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

- BertLayer
    - attention: BertAttention
        - self: BertSelfAttention
        - output: BertSelfOutput
    - intermediate: BertIntermediate, 768=>4*768
    - output: BertOutput, 4*768 => 768

In [6]:
test_sent = 'this is a test sentence'

model_input = tokenizer(test_sent, return_tensors='pt')

## 1. model output 

In [7]:
model.eval()
with torch.no_grad():
    output = model(**model_input)

In [8]:
# embeddings
output[2][0]

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [-0.6485,  0.6739, -0.0932,  ...,  0.4475,  0.6696,  0.1820],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [ 0.6010, -0.6970, -0.2001,  ...,  0.2960,  0.2060, -1.7181],
         [ 0.8323,  0.2878,  0.0021,  ...,  0.2628, -1.1310, -1.2708],
         [-0.1481, -0.2948, -0.1690,  ..., -0.5009,  0.2544, -0.0700]]])

In [9]:
# first bert layer output
output[2][1]

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]])

## 2. from scratch

- BertLayer
    - attention: BertAttention
        - self: BertSelfAttention
        - output: BertSelfOutput
    - intermediate: BertIntermediate, 768=>4\*768
    - output: BertOutput

In [12]:
embeddings = output[2][0]

In [11]:
layer = model.encoder.layer[0]

### 2.1 第一次 add & norm，发生在 mha 内部

In [13]:
mha_output = layer.attention.self(embeddings)

In [14]:
mha_output

(tensor([[[ 0.2979,  0.0801, -0.0037,  ..., -0.0142,  0.1290,  0.0828],
          [ 0.3935,  0.1356, -0.0920,  ...,  0.0211,  0.1677,  0.0011],
          [ 0.1696,  0.1449, -0.1039,  ...,  0.1604,  0.2172,  0.0310],
          ...,
          [-0.0617,  0.1968, -0.0669,  ...,  0.1126,  0.1933, -0.0204],
          [-0.2835,  0.1495, -0.0021,  ...,  0.0973,  0.1865, -0.0636],
          [ 0.2575,  0.1120, -0.1008,  ...,  0.0175,  0.1508,  0.0878]]],
        grad_fn=<ViewBackward0>),)

In [15]:
attn_output = layer.attention.output(mha_output[0], embeddings)

### 2.2 第一次 add & norm，发生在 mlp 内部

In [16]:
mlp1 = layer.intermediate(attn_output)

In [17]:
mlp1.shape

torch.Size([1, 7, 3072])

In [18]:
mlp2 = layer.output(mlp1, attn_output)

In [19]:
mlp2

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]],
       grad_fn=<NativeLayerNormBackward0>)